In [1]:
import struct
import os
from Crypto.Random import get_random_bytes
import keyboard

RC5 — это симметричный блочный шифр, основанный на операции XOR и циклических сдвигах. Циклические сдвиги перемешивают биты данных, обеспечивая диффузию.


_key_schedule

- Создается исходный ключ (например, строка из символов), а система преобразует его в массив чисел, называемый раундовыми ключами. Они будут использоваться в каждом этапе шифрования и расшифрования. Этот процесс гарантирует, что даже малейшее изменение исходного ключа (например, одна буква) полностью изменит итоговые раундовые ключи. Это делает взлом алгоритма очень сложным.

Шифрование — это процесс превращения понятного текста (открытого текста) в зашифрованный (шифротекст). Это делается с помощью раундов (циклов), где данные постепенно запутываются.

Шаги шифрования:
1. Инициализация блоков: 
    - Данные разбиваются на пары чисел (левый блок A и правый блок B).
    - К каждому блоку прибавляется по одному ключу из массива S, чтобы их запутать. 

2. Раунды: 

Для каждого раунда происходит два типа операций:
- Циклические сдвиги: Числа A и B "вращаются" на несколько бит влево.
- Сложение с ключами: Вращённые числа смешиваются с раундовыми ключами.

3. Результат: После выполнения всех раундов получается шифротекст (зашифрованный набор блоков).

In [2]:
class RC5:
    # Инициализация шифра, установка параметров.
    def __init__(self, key: bytes, word_size=32, rounds=12):
        self.word_size = word_size  # Количество бит в одном слове. Обычно это 16, 32 или 64.
        self.rounds = rounds  # Количество раундов
        self.key = key # Размер ключа
        self.modulus = 2 ** word_size  # Модуль для операций
        self.S = self._key_schedule()  # Раундовые ключи

    def _key_schedule(self):
        """Генерация раундовых ключей."""
        # Константы, используемые для инициализации массива раундовых ключей (S):
        P = 0xB7E15163 # значение, связанное с экспонентой (e).
        Q = 0x9E3779B9 # значение, связанное с золотым сечением.
        c = max(1, len(self.key) // (self.word_size // 8)) # Количество слов в ключе.
        L = [0] * c # Массив ключей.
        # L - массив, полученный из пользовательского ключа. Урезается или дополняется 
        # до нужного размера.

        # Заполняем массив L из ключа
        for i in range(len(self.key)):
            L[i // 4] = (L[i // 4] << 8) + self.key[i]

        # Генерация массива S
        S = [(P + i * Q) % self.modulus for i in range(2 * self.rounds + 2)]
        # S - массив раундовых ключей. Генерируется с использованием P и Q, а затем 
        # смешивается с помощью циклических сдвигов (_rotate_left)

        # Протокол смешивания
        A = B = i = j = 0
        for _ in range(3 * max(c, 2 * self.rounds + 2)):
            A = S[i] = self._rotate_left((S[i] + A + B) % self.modulus, 3)
            B = L[j] = self._rotate_left((L[j] + A + B) % self.modulus, (A + B) % self.word_size)
            i = (i + 1) % len(S)
            j = (j + 1) % len(L)
        return S

    # Циклические сдвиги влево и вправо используется для перемешивания данных.
    def _rotate_left(self, value, shift):
        """Циклический сдвиг влево."""
        return ((value << shift) & (self.modulus - 1)) | (value >> (self.word_size - shift))

    def _rotate_right(self, value, shift):
        """Циклический сдвиг вправо."""
        return ((value >> shift) | (value << (self.word_size - shift))) & (self.modulus - 1)

    def encrypt_block(self, block):
        """Шифрование одного блока."""
        A, B = struct.unpack("<II", block) # Делим блок на два слова.
        A = (A + self.S[0]) % self.modulus  # Добавляем начальные раундовые ключи.
        B = (B + self.S[1]) % self.modulus
        for i in range(1, self.rounds + 1):
            # Циклический сдвиг на количество бит, зависящее от значения противоположного слова и
            # сложение с очередным раундовым ключом:
            A = (self._rotate_left((A ^ B), B % self.word_size) + self.S[2 * i]) % self.modulus
            B = (self._rotate_left((B ^ A), A % self.word_size) + self.S[2 * i + 1]) % self.modulus
        return struct.pack("<II", A, B) # XOR между словами

    def decrypt_block(self, block):
        """Расшифрование одного блока."""
        A, B = struct.unpack("<II", block)
        for i in range(self.rounds, 0, -1):
            # Циклический сдвиг вправо.
            B = self._rotate_right((B - self.S[2 * i + 1]) % self.modulus, A % self.word_size) ^ A
            A = self._rotate_right((A - self.S[2 * i]) % self.modulus, B % self.word_size) ^ B
        # Вычитание раундового ключа.
        B = (B - self.S[1]) % self.modulus
        A = (A - self.S[0]) % self.modulus
        return struct.pack("<II", A, B) # XOR для восстановления оригинальных данных

Шифрование блоками требует дополнения данных до кратности размеру блока. Для этого используются различные схемы дополнения:

- ANSI: Дополняет нулями.
- X.923: Дополняет нулями и добавляет последний байт с длиной дополнения.
- PKCS7: Дополняет данными, равными количеству байтов дополнения.
- ISO7816-4: Дополняет данным байтом 0x80, затем нулями.

In [3]:
# Режимы дополнения
def pad(data, block_size, mode):
    padding_size = block_size - len(data) % block_size
    if mode == "ANSI":
        # Добавляем padding_size нулевых байтов (\x00).
        return data + b"\x00" * padding_size 
    elif mode == "X.923":
        # Добавляем (padding_size - 1) нулевых байтов, а в последний байт записывает размер дополнения.
        return data + b"\x00" * (padding_size - 1) + struct.pack("B", padding_size) 
    elif mode == "PKCS7":
        # Добавляем padding_size байтов, каждый из которых содержит значение padding_size. 
        return data + struct.pack("B", padding_size) * padding_size
    elif mode == "ISO7816-4":
        # Добавляем байт \x80, за которым следуют (padding_size - 1) нулевых байтов.
        return data + b"\x80" + b"\x00" * (padding_size - 1)
    else:
        raise ValueError("\nНеизвестный режим дополнения")

def unpad(data, block_size, mode):
    if mode == "ANSI":
        return data.rstrip(b"\x00")
    elif mode == "X.923":
        padding_size = data[-1]
        return data[:-padding_size]  # Убираем padding_size байтов
    elif mode == "PKCS7":
        padding_size = data[-1]
        return data[:-padding_size]
    elif mode == "ISO7816-4":
        if data[-1] == 0x80:
            return data.rstrip(b"\x00").rstrip(b"\x80")
        else:
            raise ValueError("Некорректное дополнение для режима ISO7816-4")
    else:
        raise ValueError("\nНеизвестный режим дополнения")

Алгоритм поддерживает несколько режимов работы, каждый из которых определяет, как блочные данные шифруются или расшифровываются:

- ECB (Electronic Codebook): Простейший режим, где каждый блок данных шифруется независимо, что не безопасно для структурированных данных (например, изображений).
- CBC (Cipher Block Chaining): Каждый блок шифруется с использованием предыдущего блока. Повышает безопасность, так как одинаковые блоки исходных данных шифруются по-разному.
- PCBC (Propagating Cipher Block Chaining): Модификация CBC, где ошибки в одном блоке влияют на расшифровку других блоков.
- CFB (Cipher Feedback): Используется для шифрования с поблочной обратной связью.
- OFB (Output Feedback): Обратный процесс, где выход предыдущего блока используется для шифрования следующего.
- CTR (Counter Mode): Использует счетчик для генерации потока ключей, который затем XOR'ится с данными. Потоковый режим, симметричный для шифрования и расшифровки.

In [4]:
# Блочные режимы
def ecb_mode(rc5, data, encrypt=True, padding_mode="PKCS7"):
    """Режим ECB (Electronic Codebook).""" 
    block_size = 8  # Данные делятся на блоки по 8 байт
    blocks = [data[i:i + block_size] for i in range(0, len(data), block_size)]
    result = b""
    
    for block in blocks:
        if len(block) < block_size:
            block = pad(block, block_size, padding_mode)  # Применяем дополнение, если блок меньше требуемого размера
        result += rc5.encrypt_block(block) if encrypt else rc5.decrypt_block(block) # шифрование
    
    return unpad(result, block_size, padding_mode) if not encrypt else result

def cbc_mode(rc5, data, iv, encrypt=True, padding_mode="PKCS7"):
    """Режим CBC (Cipher Block Chaining).""" 
    block_size = 8  # RC5 блоки по 8 байт
    blocks = [data[i:i + block_size] for i in range(0, len(data), block_size)]
    result = b""
    previous_block = iv  # Начальная синхропосылка
    
    # Каждый блок XOR'ится с результатом шифрования предыдущего блока перед шифрованием. 
    # Для первого блока используется вектор инициализации (IV).
    for block in blocks:
        if len(block) < block_size:
            block = pad(block, block_size, padding_mode)
        
        if encrypt:
            # Шифруем в режиме CBC
            # Перед шифрованием каждый блок данных объединяется (XOR) с предыдущим шифрованным блоком (previous_block).
            encrypted_block = rc5.encrypt_block(bytes(a ^ b for a, b in zip(block, previous_block)))
            result += encrypted_block
            previous_block = encrypted_block  # Обновляем предыдущий блок
        else:
            # Расшифровка в режиме CBC
            # Для расшифровки используется тот же принцип, только XOR выполняется после расшифровки текущего блока.
            decrypted_block = rc5.decrypt_block(block)
            result += bytes(a ^ b for a, b in zip(decrypted_block, previous_block))
            previous_block = block  # Обновляем предыдущий блок
    
    return unpad(result, block_size, padding_mode) if not encrypt else result

def pcbc_mode(rc5, data, iv, encrypt=True, padding_mode="PKCS7"):
    """Режим PCBC (Propagating Cipher Block Chaining)."""
    # Похож на CBC, но при шифровании (или расшифровке) XOR выполняется как с предыдущим блоком 
    # данных, так и с предыдущим зашифрованным блоком.
    block_size = 8  # RC5 блоки по 8 байт
    blocks = [data[i:i + block_size] for i in range(0, len(data), block_size)]
    result = b""
    previous_block = iv  # Начальная синхропосылка

    for block in blocks:
        if len(block) < block_size:
            block = pad(block, block_size, padding_mode)
        
        if encrypt:
            # Шифруем в режиме PCBC
            # XOR выполняется с текущим блоком данных и предыдущим результатом шифрования.
            xor_block = bytes(a ^ b for a, b in zip(block, previous_block))
            encrypted_block = rc5.encrypt_block(xor_block)
            result += encrypted_block
            previous_block = encrypted_block  # Обновляем предыдущий блок
        else:
            # Расшифровка в режиме PCBC
            # Для расшифровки аналогично используется результат расшифровки и предыдущий блок.
            decrypted_block = rc5.decrypt_block(block)
            xor_block = bytes(a ^ b for a, b in zip(decrypted_block, previous_block))
            result += xor_block
            previous_block = block  # Обновляем предыдущий блок
    
    return unpad(result, block_size, padding_mode) if not encrypt else result

def cfb_mode(rc5, data, iv, encrypt=True, padding_mode="PKCS7"):
    """Режим CFB (Cipher Feedback)."""
    # Блочный шифр используется для создания псевдослучайной последовательности, которая XOR'ится с данными.
    blocks = [data[i:i + 8] for i in range(0, len(data), 8)]
    result = b""
    feedback = iv
    
    # IV шифруется, и результат XOR'ится с блоком данных.
    # В следующем цикле вместо данных используется предыдущий выход XOR для создания псевдослучайности.    
    for block in blocks:
        feedback = rc5.encrypt_block(feedback)
        if encrypt:
            # XOR исходных данных с результатом шифрования feedback
            block = pad(block, 8, padding_mode) if len(block) < 8 else block
            xor_output = bytes(a ^ b for a, b in zip(block, feedback))
            result += xor_output
            feedback = xor_output
        else:
            # Используется исходный зашифрованный текст вместо feedback
            xor_output = bytes(a ^ b for a, b in zip(block, feedback))
            result += xor_output
            feedback = block

    return unpad(result, 8, padding_mode) if not encrypt else result

def ofb_mode(rc5, data, iv):
    """Режим OFB (Output Feedback)."""
    # Похож на CFB, но вместо использования зашифрованных данных используется зашифрованный IV
    blocks = [data[i:i + 8] for i in range(0, len(data), 8)]
    result = b""
    feedback = iv
    
    # Каждый раз шифруется feedback (начальный IV).
    for block in blocks:
        feedback = rc5.encrypt_block(feedback)
        # XOR с блоком данных для получения результата.
        xor_output = bytes(a ^ b for a, b in zip(block, feedback))
        result += xor_output

    return result

def ctr_mode(rc5, data, iv):
    """Режим CTR (Counter Mode)."""
    # Использует счетчик, который шифруется для создания псевдослучайного ключа, XOR'ящегося 
    # с блоком данных. Обеспечивает параллельную обработку данных, так как каждый блок независим.
    block_size = 8  # Размер блока RC5 (64 бита = 8 байт)
    blocks = [data[i:i + block_size] for i in range(0, len(data), block_size)]
    result = b""

    # Инициализация значения счетчика
    counter = struct.unpack("<Q", iv)[0]  # IV задает начальное значение счетчика

    for block in blocks:
        # Шифруем текущее значение счетчика
        counter_block = struct.pack("<Q", counter)
        encrypted_counter = rc5.encrypt_block(counter_block)

        # XOR блока данных с зашифрованным счетчиком
        xor_output = bytes(a ^ b for a, b in zip(block, encrypted_counter))
        result += xor_output

        # Увеличиваем счетчик (по модулю 2^64, так как длина IV — 64 бита)
        counter = (counter + 1) % (2**64)

    return result


In [5]:
def process_file(input_file, output_file, key_file, mode, iv_file=None, encrypt=True, padding="PKCS7"):
    """Обрабатывает файл в соответствии с выбранными настройками."""
    with open(input_file, "rb") as f:
        data = f.read()
    with open(key_file, "rb") as f:
        key = f.read()

    iv = b""
    if iv_file:
        with open(iv_file, "rb") as f:
            iv = f.read()

    rc5 = RC5(key)

    if mode == "ECB":
        result = ecb_mode(rc5, data, encrypt, padding)
    elif mode == "CBC":
        result = cbc_mode(rc5, data, iv, encrypt, padding)
    elif mode == "PCBC":
        result = pcbc_mode(rc5, data, iv, encrypt, padding)
    elif mode == "CFB":
        result = cfb_mode(rc5, data, iv, encrypt, padding)
    elif mode == "OFB":
        result = ofb_mode(rc5, data, iv)
    elif mode == "CTR":
        result = ctr_mode(rc5, data, iv)
    else:
        raise ValueError(f"\nНеподдерживаемый режим: {mode}")

    with open(output_file, "wb") as f:
        f.write(result)

In [6]:
def generate_binary_file(filename, size):
    """
    Генерирует файл с бинарными данными заданного размера.
    
    :param filename: Имя файла для записи данных.
    :param size: Размер данных в байтах.
    """
    with open(filename, "wb") as f:
        f.write(os.urandom(size))
    print(f"\nФайл '{filename}' с размером {size} байт успешно создан.")

def generate_test_files():
    """
    Генерирует набор входных файлов:
    - `in.bin`: файл с открытым текстом.
    - `key.bin`: файл с ключом.
    - `sync.bin`: файл с синхропосылкой (IV).
    """
    input_size = 64  # Размер открытого текста (512 бита = 64 байта)
    key_size = 16    # Размер ключа (128 бит = 16 байт)
    iv_size = 8      # Размер IV (64 бита = 8 байт)
    
    # Генерация файлов
    with open("in.bin", "wb") as infile:
        infile.write(b"Hello, this is a test message for DES encryption! ")
    with open("key.bin", "wb") as keyfile:
        key = get_random_bytes(key_size) # Генерация ключа.
        print("Ключ: " + str(key.hex()))
        keyfile.write(key)
        #keyfile.write(b"0x2b7e151628aed2a6abf7158809cf4f3c")
    with open("sync.bin", "wb") as ivfile:
        iv = get_random_bytes(iv_size)  # Генерация синхропосылки.
        print("Cинхропосылка: " + str(iv.hex()))
        ivfile.write(iv)

# Генерация файлов
generate_test_files()


Ключ: aa5f89cb3c5abf122469ff56267925a1
Cинхропосылка: c2171c340c557af7


In [7]:
def select_from_list(prompt, options):
    """
    Функция для выбора элемента из списка через консоль.
    
    :param prompt: Строка приглашения для выбора.
    :param options: Список возможных вариантов.
    :return: Индекс выбранного элемента.
    """
    while True:
        if keyboard.is_pressed('esc'):
            print("\nПрограмма завершена.")
            return None
        try:
            choice = int(input(prompt))
            if 0 <= choice < len(options):
                return choice
            else:
                print("\nОшибка: номер вне диапазона.")
        except ValueError:
            print("\nОшибка: введите число.")

In [8]:
def main():
    modes = ["ECB", "CBC", "PCBC", "CFB", "OFB", "CTR"]
    paddings = ["ANSI", "X.923", "PKCS7", "ISO/IEC 7816-4"]
    operations = ["Зашифровать", "Расшифровать"]

    print("Режимы шифрования:")
    for i, mode in enumerate(modes, 0):
        print(f"{i}. {mode}")

    print("\nРежимы дополнения:")
    for i, padding in enumerate(paddings, 0):
        print(f"{i}. {padding}")

    print("\nДоступные операции:")
    for i, operation in enumerate(operations, 0):
        print(f"{i}. {operation}")

    # Запросить у пользователя пути к файлам
    input_file = input("Введите путь к файлу с входными данными (например, in.bin): ")
    output_file = input("Введите путь для выходного файла (например, out.bin): ")
    #key_file = input("Введите путь к файлу с ключом (например, key.bin): ")
    #iv_file = input("Введите путь к файлу с синхропосылкой (если не требуется, нажмите Enter): ") or None
    key_file = "key.bin"
    iv_file = "sync.bin"

    # Выбрать режим шифрования
    mode_index = select_from_list("\nВыберите режим шифрования: ", modes)
    mode = modes[mode_index]

    # Выбрать режим дополнения
    padding_index = select_from_list("\nВыберите режим дополнения: ", paddings)
    padding = paddings[padding_index]

    # Выбрать операцию
    operation_index = select_from_list("\nВыберите операцию:", operations)
    encrypt = operation_index == 0  # True для "Зашифровать", False для "Расшифровать"

    # Выполнить обработку
    try:
        process_file(
            input_file=input_file,
            output_file=output_file,
            key_file=key_file,
            mode=mode,
            iv_file=iv_file,
            encrypt=encrypt,
            padding=padding,
        )
        print(f"\nОперация '{operations[operation_index]}' успешно выполнена. Результат записан в '{output_file}'.")
    except Exception as e:
        print(f"\nОшибка при обработке файла: {e}")

# Запуск программы
#main()


In [9]:
def automated_testing():
    modes = ["ECB", "CBC", "PCBC", "CFB", "OFB", "CTR"]
    paddings = ["ANSI", "X.923", "PKCS7", "ISO7816-4"]
    input_file = "in.bin"
    encrypted_file = "out.bin"
    decrypted_file = "decrypted.bin"
    key_file = "key.bin"
    iv_file = "sync.bin"

    # Чтение исходного файла
    with open(input_file, "rb") as f:
        original_data = f.read()

    # Чтение ключа
    with open(key_file, "rb") as f:
        key = f.read()

    # Чтение синхропосылки (IV)
    with open(iv_file, "rb") as f:
        iv = f.read()

    for mode in modes:
        for padding in paddings:
            print(f"\n=== Режим шифрования: {mode}, режим дополнения: {padding} ===")
            try:
                # Создаем экземпляр RC5
                rc5 = RC5(key)

                # Шифрование
                if mode == "ECB":
                    encrypted_data = ecb_mode(rc5, original_data, encrypt=True)
                elif mode == "CBC":
                    encrypted_data = cbc_mode(rc5, original_data, iv, encrypt=True)
                elif mode == "PCBC":
                    encrypted_data = pcbc_mode(rc5, original_data, iv, encrypt=True)
                elif mode == "CFB":
                    encrypted_data = cfb_mode(rc5, original_data, iv, encrypt=True)
                elif mode == "OFB":
                    encrypted_data = ofb_mode(rc5, original_data, iv)
                elif mode == "CTR":
                    encrypted_data = ctr_mode(rc5, original_data, iv)
                else:
                    continue

                # Сохраняем зашифрованные данные
                with open(encrypted_file, "wb") as f:
                    f.write(encrypted_data)

                # Расшифрование
                if mode == "ECB":
                    decrypted_data = ecb_mode(rc5, encrypted_data, encrypt=False)
                elif mode == "CBC":
                    decrypted_data = cbc_mode(rc5, encrypted_data, iv, encrypt=False)
                elif mode == "PCBC":
                    decrypted_data = pcbc_mode(rc5, encrypted_data, iv, encrypt=False)
                elif mode == "CFB":
                    decrypted_data = cfb_mode(rc5, encrypted_data, iv, encrypt=False)
                elif mode == "OFB":
                    decrypted_data = ofb_mode(rc5, encrypted_data, iv)
                elif mode == "CTR":
                    decrypted_data = ctr_mode(rc5, encrypted_data, iv)
                else:
                    continue

                # Сохраняем расшифрованные данные
                with open(decrypted_file, "wb") as f:
                    f.write(decrypted_data)

                # Вывод содержимого файлов
                print("Исходные данные:      ", original_data)
                print("Зашифрованные данные: ", encrypted_data)
                print("Дешифрованные данные: ", decrypted_data)

                # Проверка совпадения с учетом удаления дополнения
                if original_data == decrypted_data:
                    print("УСПЕХ: Исходные и дешифрованные данные совпадают.")
                else:
                    print("ОШИБКА: Исходные и дешифрованные данные не совпадают.")

            except Exception as e:
                print(f"Ошибка шифрования с режимом {mode} и режимом дополнения {padding}: {e}")

# Запуск автоматического тестирования
automated_testing()



=== Режим шифрования: ECB, режим дополнения: ANSI ===
Исходные данные:       b'Hello, this is a test message for DES encryption! '
Зашифрованные данные:  b'\xb9\xe4\x87\x10\xa1\xa49\x0c\xf8\xe9\x03\xd2z\x9d\xd1\x7f\\\x13\xaf8\xd0c\xf1>\xd1\xe7C\x03\xaa3\x17>\x1b\xb8H-\x16\xb5\x87v\x8a_Z\x10\xec\xa4\xc3\x18\xcdC\xa2\xd2\xb8\xdcSt'
Дешифрованные данные:  b'Hello, this is a test message for DES encryption! '
УСПЕХ: Исходные и дешифрованные данные совпадают.

=== Режим шифрования: ECB, режим дополнения: X.923 ===
Исходные данные:       b'Hello, this is a test message for DES encryption! '
Зашифрованные данные:  b'\xb9\xe4\x87\x10\xa1\xa49\x0c\xf8\xe9\x03\xd2z\x9d\xd1\x7f\\\x13\xaf8\xd0c\xf1>\xd1\xe7C\x03\xaa3\x17>\x1b\xb8H-\x16\xb5\x87v\x8a_Z\x10\xec\xa4\xc3\x18\xcdC\xa2\xd2\xb8\xdcSt'
Дешифрованные данные:  b'Hello, this is a test message for DES encryption! '
УСПЕХ: Исходные и дешифрованные данные совпадают.

=== Режим шифрования: ECB, режим дополнения: PKCS7 ===
Исходные данные:       